In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns




In [3]:
df=pd.read_csv('https://raw.githubusercontent.com/isaavedrab85/PROYECTO/refs/heads/main/PROYECTO/01database.csv',sep=';')


In [4]:
df.head() # visualización primeras columnas
print(df)
df.shape # tamaño del dataset (filas, columnas)
df.info() # información del dataset
df.describe() # estadísticas del dataset





     RUT SEXO  TIPO COL          COMUNA  P_POND  P_PROM  P_NEM  RK_COL  P_LEN  \
0      1    M         1       PEÑALOLEN  788.70   823.0    771     771    729   
1      2    M         2    SAN BERNARDO  709.25   682.5    785     813    669   
2      3    M         2      LA FLORIDA  683.00   638.0    779     835    607   
3      4    F         3           AYSEN  776.15   680.5    928    1000    656   
4      5    M         2       QUILICURA  858.70   858.0    900     917    799   
..   ...  ...       ...             ...     ...     ...    ...     ...    ...   
608  609    M         2     PUENTE ALTO  651.90   608.0    752     804    600   
609  610    M         3       EL BOSQUE  544.50   620.5    523     523    625   
610  611    M         2    SAN BERNARDO  572.20   582.5    613     613    611   
611  612    M         2        RECOLETA  566.90   602.5    538     538    563   
612  613    M         2  ISLA DE PASCUA  578.20   502.0    713     713    481   

     P_M1  ...  C2_24-2  AY

,RUT,TIPO COL,P_POND,P_PROM,P_NEM,RK_COL,P_LEN,P_M1,POND_SIN_M2,P_M2,...,C2_24-2,AYG_24-2,C1_24-2,AP_AYG,AP_CI,AP_AL,AP_C2,AP_AYG_2,AP_CI_2,Total
count,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,...,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000,613.000000
mean,307.000000,2.091354,705.314519,700.242251,749.657423,774.440457,674.528548,725.955954,683.165987,442.970636,...,3.853311,0.658418,0.514869,77487.765090,8189.233279,688.417618,67.047308,1.076672,0.079935,86433.619902
std,177.102136,0.572924,68.437849,83.492038,111.307632,128.097958,90.987580,104.960324,66.690282,80.165387,...,2.175082,1.538108,1.383147,41800.396342,3853.961746,463.518381,47.042527,3.102128,0.271414,44948.986282
min,1.000000,0.000000,503.000000,426.500000,356.000000,364.000000,375.000000,452.000000,481.600000,246.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,154.000000,2.000000,654.300000,640.500000,680.000000,685.000000,600.000000,657.000000,635.550000,393.000000,...,2.600000,0.000000,0.000000,100000.000000,10000.000000,0.000000,0.000000,0.000000,0.000000,101001.000000
50%,307.000000,2.000000,711.900000,695.500000,757.000000,782.000000,679.000000,717.000000,690.800000,437.000000,...,4.600000,0.000000,0.000000,100000.000000,10000.000000,1000.000000,100.000000,0.000000,0.000000,111100.000000
75%,460.000000,2.000000,751.550000,758.000000,837.000000,872.000000,738.000000,797.000000,729.700000,487.000000,...,5.325000,0.000000,0.000000,100000.000000,10000.000000,1000.000000,100.000000,0.000000,0.000000,111100.000000
max,613.000000,4.000000,919.050000,907.000000,1000.000000,1000.000000,935.000000,990.000000,885.950000,718.000000,...,7.000000,5.975000,6.080000,100000.000000,10000.000000,1000.000000,100.000000,10.000000,1.000000,111100.000000


#Detalles 01DATABASE

Las colulmnas de mi archivo "01database.csv" de base de datos contienen los ID y puntajes variados de pruebas de admisión PAES, diagnóstico y cursos de matemáticas del primer año.

El detalle de cada columna es el siguiente:

### Columna 1: ID de cada estudiante.
### Columna 2: Sexo
*   M = Masculino
*   F = Femenino

### Columna 3: Tipo de Colegio
*   1 = Particular
*   2 = Subencionado
*   3 = Municipal
*   4 = Bicentenerio

### Columna 4: Comuna del colegio.

### Columna 5: Puntaje Ponderado PAES para carrera Ingenieria
$$P_P = \text{NEM} \times 0,1 + P_{Rk}\times 0,3 + P_{CL}\times 0,1+ P_{M_1}\times 0,35 + P_{M_2}\times 0,05 + (P_H\quad o \quad P_C)\times 0,1.$$ Este puntaje va de 100 a 1000 y depende de cada la ponderación que se da a cada variable y que permite ordenar a los postulantes para su posterior matrícula.

### Columna 6: Puntaje Promedio $(P_M)$: $$P_M = \dfrac{P_{M_1} + P_{CL}}{2}.$$ Este puntaje va de 100 a 1000.

Es el promedio de las pruebas de Matemáticas 1 y Comprensión Lectora. Este puntaje va de 100 a 1000.

### Columna 7: Puntaje Notas Enseñanza Media $(NEM)$.

Este es el puntaje que se asigna al promedio de notas obtenido en los 4 años de enseñanza media. Este puntaje va de 100 a 1000.

### Columna 8: Puntaje Ranking $(P_{RK})$.

Este es el puntaje que se asigna a cada estudiante de acuerdo a la posición relativa de sus notas, en la enseñanza media, en comparación con las notas de tres generaciones de alumnos de su mismo colegio. Este puntaje va de 100 a 1000.

### Columna 9: Puntaje Comprensión Lectora $(P_{CL})$.

Es el puntaje obtenido por el estudiante en la PAES de Comprensión Lectora.  Este puntaje va de 100 a 1000.

### Columna 10: Es el puntaje Ponderado sin la PAES M2.

### Columna 11: Puntaje Matemáticas 1 $(P_{M_{1}})$.
Este puntaje va de 100 a 1000.

Es el puntaje obtenido por el estudiante en la PAES de Matemáticas 1. Este puntaje va de 100 a 1000.

### Columna 12: Puntaje Matemáticas 2 $(P_{M_{2}})$.

Es el puntaje obtenido por el estudiante en la PAES de Matemáticas 2. Este puntaje va de 100 a 1000.

### Columna 13: Puntaje Historia $(P_{H})$.

Es el puntaje obtenido por el estudiante en la PAES de Historia. Este puntaje va de 100 a 1000.

### Columna 14: Puntaje Ciencias $(P_{C})$.

Es el puntaje obtenido por el estudiante en la PAES de Ciencias. Este puntaje va de 100 a 1000.

### Columna 15: Máximo puntaje entre $(P_{H})$ y $(P_{C})$.


### Columna 16: Puntaje (%) Diagnóstico $(P_{Diag})$.

Es el porcentaje de logro obtenido en la prueba de Diagnóstico de Matemáticas que se realiza a los estudiantes nuevos. Este puntaje va de 0 a 1.


### Columna 17: Puntaje Álgebra y Geometría 2024-1 $(P_{A\& G})$.

Es la nota obtenida en el curso de Álgebra y Geometría. Esta va de 1 a 7.

### Columna 18: Puntaje Cálculo I 2024-1 $(P_{CI})$.

Es la nota obtenida en el curso de Cálculo I. Esta va de 1 a 7.

### Columna 19: Puntaje Álgebra Lineal 2024-2 $(P_{AL})$.

Es la nota obtenida en el curso de Álgebra Lineal. Este curso se puede rendir sólo si se aprueba el curso de Álgebra y Geometría el primer semestre. Su valor va de 1 a 7.


### Columna 20: Puntaje Cálculo II 2024-2 $(P_{AL})$.

Es la nota obtenida en el curso de Cálculo II. Este curso se puede rendir sólo si se aprueba el curso de Cálculo I el primer semestre. Su valor va de 1 a 7.

### Columna 21: Puntaje Álgebra y Geometría 2024-2 (repetición) $(\widetilde{P_{AL}})$. Es la nota obtenida en el curso de Álgebra y Geometría de repetición. Este curso es rendido por estudiantes que reprobaron en el primer semestre este mismo curso. Su valor va de 1 a 7.



### Columna 22: Puntaje Cálculo I 2024-2 (repetición) $(\widetilde{P_{CI}})$.

Es la nota obtenida en el curso de Cálculo I de repetición. Este curso es rendido por estudiantes que reprobaron en el primer semestre este mismo curso. Su valor va de 1 a 7.

### Columna 23: Etiqueta con un 100000 si el estudiante aprobó el curso AYG_24-1, en caso contrario etiqueta con 0.

### Columna 24: Etiqueta con un 10000 si el estudiante aprobó el curso CI_24-1, en caso contrario etiqueta con 0.

### Columna 25: Etiqueta con un 1000 si el estudiante aprobó el curso AL_24-2, en caso contrario etiqueta con 0.

### Columna 26: Etiqueta con un 100 si el estudiante aprobó el curso CII_24-2, en caso contrario etiqueta con 0.

### Columna 27: Etiqueta con un 10 si el estudiante reprobó AYG_24-1, pero aprobó el curso AYG_24-2, en caso contrario etiqueta con 0.

### Columna 28: Etiqueta con un 1 si el estudiante reprobó CI_24-1, pero aprobó el curso CI_24-2, en caso contrario etiqueta con 0.

### Columna 29: Suma las columnas 23,24,25,26,27 y 28. Esto me permite identificar si un estudiante aprobó o reprobó cuantos y qué cursos.